# Event class

**An event is a simple concurrency primitive that allows communication between threads**

*This is one of the simplest mechanisms for communication between threads: one thread signals an event and other threads wait for it.*

In [2]:
from threading import Thread, Event, currentThread

import time

def f(e):
    print('{} is waiting for event.'.format(currentThread().getName()))
    value = e.wait()
    print('{} finished waiting.'.format(currentThread().getName()))

def g(e):
    time.sleep(1)
    print('{} sets the event.'.format(currentThread().getName()))
    e.set()

e = Event()

# **e.wait()**
It locks current thread and wait for e.set() to be called.

# **e.set()**
It sets free current thread and continue its job

In [3]:
t = Thread(name='t', target=f, args=(e,))
s = Thread(name='s', target=g, args=(e,))

t.start()
s.start()

t is waiting for event.


s sets the event.
t finished waiting.


# **Event.wait() has a "time out" argument which has two condition for itself**

*First one, As we saw it waits for set function to be called and after that it set True value for wait function and continue its job*

*Second one, If the timeout value is much more than the value we set in e.wait() function it set False value for wait function and somehow ignore it.*

In [6]:
from threading import Thread, Event, currentThread

import time

def f(e):
    print('{} is waiting for event.'.format(currentThread().getName()))
    value = e.wait(2)
    if value:
        print('other thread set the event')
    else:
        print('timeout finished')

def g(e, t):
    time.sleep(t)
    e.set()
    print('{} set the event.'.format(currentThread().getName()))

In [5]:
#timeout check

e = Event()

t = Thread(name='t', target=f, args=(e,))
s = Thread(name='s', target=g, args=(e,3))

t.start()
s.start()

t.join()
print('event value is {}'.format(e.isSet()))
s.join()

t is waiting for event.
timeout finished
event value is False
s set the event.


*Don't forget with **e.clear()** we could change the condition for thread to its default*

And **e.isSet()** return True if e.wait() value has been True.

In [7]:
#regular check

e.clear()

t = Thread(name='t', target=f, args=(e,))
s = Thread(name='s', target=g, args=(e,1))

t.start()
s.start()

t.join()
print('event value is {}'.format(e.isSet()))
s.join()

t is waiting for event.
s set the event.other thread set the event

event value is True


# Lock Class

This lock helps us in the synchronization of two or more threads. Lock class perhaps provides the simplest synchronization primitive in Python.

** Any Lock object has two conditon: **

1. *Locked* 
2. *Unlocked*

**By calling acquire() function one these two happens:**

1. If the object is locked it(thread) waits for object to be unlocked and after that it locks the object and continue its job.
2. But if the object is unlocked after calling function Lock function it locks and thread continue its job.

**For locking an object you can use release() function but pay attention if your object is locked before calling release function it return Error.**

In [4]:
from threading import Thread, Lock

class Counter:
    def __init__(self):
        self.lock = Lock()
        self.value = 0

    def increment(self):
        self.lock.acquire()
        self.value += 1
        self.lock.release()

def f():
    global counter
    for i in range(200000):
        counter.increment()

def test():
    global counter
    counter = Counter()

    t = Thread(target=f)
    s = Thread(target=f)

    t.start()
    s.start()

    t.join()
    s.join()

for i in range(1, 5):
    test()
    print("in test {}: value of a is {} and must be {}".format(i, counter.value, 200000*2))

in test 1: value of a is 400000 and must be 400000
in test 2: value of a is 400000 and must be 400000
in test 3: value of a is 400000 and must be 400000
in test 4: value of a is 400000 and must be 400000
